In [1]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt
import numpy as np
import time

In [2]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]
print(len(classes))

80


In [3]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]


In [ ]:
#Loading image
cap = cv2.VideoCapture("Busy street crossing in Hong Kong.mp4-.mp4")
font =cv2.FONT_HERSHEY_SIMPLEX
starting_time = time.time()
colors = np.random.uniform(0,255, size = (len(classes),3))
frame_id = 0

while True:
    _,frame = cap.read()
    height, width, channels = frame.shape 
    frame_id+=1
 
    #detecting object
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence>0.2:
                #object detected
                centre_x = int(detection[0]*width)
                centre_y = int (detection[1]*height)
                w  =int(detection[2]*width)
                h = int (detection[3]*height)

                #cv2.circle(img, (centre_x,centre_y), 10, (0,255,0), 2)
                #rectangle coordinates
                x = int(centre_x-w/2)
                y = int(centre_y-h/2)

                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.4)
    num_obj_detected = len(boxes)
    
    for i in range(num_obj_detected):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[i]
            cv2.rectangle(frame, (x,y), (x+w,y+h),color ,2)
            cv2.putText(frame, label, (x,y + 30), font,1,color, 3)
            cv2.putText(frame, label+ str(round(confidence,2)),(x,y + 30), font,1,color, 3)
    
    time_elapsed  = time.time() - starting_time
    fps = frame_id/time_elapsed
    cv2.putText(frame, "FPS: "+str(round(fps,2)), (10,50), font,1,(0,255,0), 1)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()   